In [39]:
import pandas as pd
import glob
import re
import os
import requests
import time
from tqdm import tqdm


df_s = pd.read_csv("article_data.csv", sep=",", header=0)
#parse out beginning page
df_s[['BP', 'EP']] = df_s['pages'].str.split('-', expand=True)
#df_s = pd.read_csv(soc10_d, sep=",", header4=0, usecols = ["journal","volume", "issue", "pages"])
df_s



,artid,journal,volume,issue,year,month,pages,cited3,cited2012,aut_num,...,bip_avgdegree,aut_flowhier,ack_flowhier,com_flowhier,aut_numcomp,ack_numcomp,com_numcomp,bip_numcomp,BP,EP
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,0.000000,99.0,99.0,99.000000,1,0,1,1,1,19
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,0.000000,99.0,99.0,99.000000,1,0,1,1,21,42
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,0.777778,1.0,0.2,0.277778,1,3,3,5,43,65
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,0.666667,99.0,99.0,0.000000,1,2,2,2,67,85
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,0.000000,0.0,99.0,0.000000,1,0,1,2,87,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,4.250000,0.0,99.0,0.000000,1,1,1,1,122,127
3594,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,0.333333,99.0,99.0,0.000000,1,5,5,5,138,167
3595,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,0.000000,1.0,99.0,1.000000,3,1,4,5,128,137
3596,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,4.250000,0.0,99.0,0.000000,1,1,1,1,96,116


In [40]:

wos_meta_d = "WoS_sociology_metadata_2000_2009/"
dfs = []
for r, d, f0 in os.walk(wos_meta_d):
    for fname in f0:
        if "txt" in fname:
            df0 = pd.read_csv( os.path.join(wos_meta_d,fname), sep='\t', header=0, dtype={"BP":str, "EP":str} )
            dfs.append(df0)
df = pd.concat(dfs)
df = df.reset_index(drop=True)
df


,PT,AU,BA,CA,GP,RI,OI,BE,Z2,AU.1,...,DG,C1,C3,EC,DE,DA,UT,PM,Unnamed: 78,Publication Type
0,J,"Hallinan, Maureen T.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258547900003,NaN,NaN,NaN
1,J,"Jasper, James M.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258833100019,NaN,NaN,NaN
2,J,"Jean, Peter K. B. St.; Weitzer, Ronald",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258833100017,NaN,NaN,NaN
3,J,"Kaminski, Elizabeth",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258833100020,NaN,NaN,NaN
4,J,"Kempner, Joanna",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258833100010,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dissertation and Thesis Subjects
6312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Author Keywords
6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Indexed Date
6314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UT (Unique ID)


In [41]:

for idx, c in enumerate(df.columns):
    print(idx+1, c)


1 PT
2 AU
3 BA
4 CA
5 GP
6 RI
7 OI
8 BE
9 Z2
10 AU.1
11 AA
12 TI
13 X1
14 Y1
15 Z1
16 FT
17 PN
18 AE
19 Z3
20 SO
21 S1
22 SE
23 BS
24 VL
25 IS
26 SI
27 MA
28 BP
29 EP
30 AR
31 VN
32 VH
33 DI
34 D2
35 L1
36 L2
37 L3
38 EA
39 SU
40 DT
41 PD
42 PY
43 AB
44 X4
45 Y4
46 Z4
47 AK
48 CT
49 CY
50 SP
51 CL
52 TC
53 Z8
54 ZR
55 ZA
56 ZB
57 ZS
58 Z9
59 U1
60 U2
61 SN
62 EI
63 BN
64 G1
65 NR
66 CR
67 LA
68 AS
69 AC
70 CG
71 DG
72 C1
73 C3
74 EC
75 DE
76 DA
77 UT
78 PM
79 Unnamed: 78
80 Publication Type


In [42]:

df1 = df[["DI", "D2", "AU","TI","SO","PY","VL","IS","BP","EP"]] # DOI, Authors, Title, Source, Publication Year, Volume, Issue, Begin Page, End Page
df1["pages"] = df1['BP'].astype(str) + '-' + df1['EP'].astype(str)
df1[df1["DI"].isna()]


<ipython-input-42-d31685ecafe3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["pages"] = df1['BP'].astype(str) + '-' + df1['EP'].astype(str)


,DI,D2,AU,TI,SO,PY,VL,IS,BP,EP,pages
2,NaN,NaN,"Jean, Peter K. B. St.; Weitzer, Ronald",Race and Policing in America: Conflict and reform,AMERICAN JOURNAL OF SOCIOLOGY,2008.0,114.0,1.0,254,256,254-256
31,NaN,NaN,"Ben-Yehuda, Nachman",The path of the devil: Early modern witch hunts,SOCIAL FORCES,2008.0,86.0,4.0,1844,1845,1844-1845
36,NaN,NaN,"Cadge, Wendy",Straight to Jesus: Sexual and Christian conver...,SOCIAL FORCES,2008.0,86.0,4.0,1841,1843,1841-1843
39,NaN,NaN,"Collins, Patricia Hill",Party/politics: Horizons in black political th...,SOCIAL FORCES,2008.0,86.0,4.0,1865,1867,1865-1867
40,NaN,NaN,"Crowley, Martha",From hire to liar: The role of deception in th...,SOCIAL FORCES,2008.0,86.0,4.0,1867,1869,1867-1869
...,...,...,...,...,...,...,...,...,...,...,...
6311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan-nan
6312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan-nan
6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan-nan
6314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan-nan


In [43]:

df_join = df_s.merge(df1, left_on = ["year","volume", "issue", "BP"], right_on=["PY","VL","IS","BP"], how="left")
df_join



,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,DI,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,10.2307/2657293,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127
3604,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,10.2307/2657295,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167
3605,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,10.2307/2657294,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137
3606,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,10.2307/2657291,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116


In [44]:
df1.groupby("SO").count()

,DI,D2,AU,TI,PY,VL,IS,BP,EP,pages
SO,,,,,,,,,,
AMERICAN JOURNAL OF SOCIOLOGY,1893,0,1989,1989,1989,1989,1978,1989,1989,1989
AMERICAN SOCIOLOGICAL REVIEW,465,0,481,481,481,481,481,481,481,481
ANNUAL REVIEW OF SOCIOLOGY,233,0,234,239,239,239,0,239,239,239
JOURNAL OF MARRIAGE AND FAMILY,922,0,1100,1100,1100,1100,1100,1092,1092,1100
JOURNAL OF SUSTAINABLE AGRICULTURE,1,0,1,1,1,1,1,1,1,1
SOCIAL FORCES,945,0,1189,1189,1189,1189,1189,1187,1187,1189
SOCIAL NETWORKS,252,0,252,252,252,252,252,252,252,252
SOCIAL SCIENCE RESEARCH,439,0,443,443,443,443,443,443,443,443
"SOCIOLOGICAL METHODOLOGY 2000, VOL 30",7,0,8,8,8,8,0,8,8,8


In [45]:
na = df_join[df_join["DI"].isna()]

In [46]:
na

,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,DI,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578,3721,socimeth,30,0,2000,6,1-28,2,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3579,3722,socimeth,30,0,2000,6,237-260,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3580,3723,socimeth,30,0,2000,6,261-315,0,12,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3581,3724,socimeth,30,0,2000,6,165-202,2,73,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
na_new = na.merge(df1, left_on = ["year","volume", "BP"], right_on=["PY","VL","BP"], how="left")
na_new

,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,DI_y,D2_y,AU_y,TI_y,SO_y,PY_y,VL_y,IS_y,EP,pages
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,608,1-608
1,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,10.1146/annurev-soc-070308-115936,NaN,"Gans, Herbert J.",Working in Six Research Areas: A Multi-Field S...,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,19,1-19
2,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,10.1146/annurev-soc-070308-115916,NaN,"Brubaker, Rogers","Ethnicity, Race, and Nationalism",ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,42,21-42
3,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,10.1146/annurev-soc-070308-115954,NaN,"Jacobs, Jerry A.; Frickel, Scott",Interdisciplinarity: A Critical Assessment,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,65,43-65
4,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,10.1146/annurev.soc.34.040507.134631,NaN,"Andersen, Robert",Nonparametric Methods for Modeling Nonlinearit...,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,85,67-85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,3721,socimeth,30,0,2000,6,1-28,2,4,3,...,NaN,NaN,"Arminger, G; Clogg, CC; Cheng, TW",Regression analysis of multivariate binary res...,"SOCIOLOGICAL METHODOLOGY 2000, VOL 30",2000.0,30.0,NaN,26,1-26
607,3722,socimeth,30,0,2000,6,237-260,0,1,1,...,10.1111/0081-1750.00080,NaN,"Hoem, JM",Systematic patterns of zero exposures in event...,"SOCIOLOGICAL METHODOLOGY 2000, VOL 30",2000.0,30.0,NaN,259,237-259
608,3723,socimeth,30,0,2000,6,261-315,0,12,1,...,10.1111/0081-1750.00081,NaN,"Montgomery, JD","The self as a fuzzy set of roles, role theory ...","SOCIOLOGICAL METHODOLOGY 2000, VOL 30",2000.0,30.0,NaN,314,261-314
609,3724,socimeth,30,0,2000,6,165-202,2,73,2,...,10.1111/0081-1750.00078,NaN,"Yuan, KH; Bentler, PM",Three likelihood-based methods for mean and co...,"SOCIOLOGICAL METHODOLOGY 2000, VOL 30",2000.0,30.0,NaN,200,165-200


In [48]:
na_new[na_new["DI_y"].isna()]

,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,DI_y,D2_y,AU_y,TI_y,SO_y,PY_y,VL_y,IS_y,EP,pages
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,608,1-608
28,28,annurevisoci,34,1,2008,6,1-13,3,3,1,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2008.0,34.0,NaN,540,1-540
51,50,annurevisoci,33,1,2007,6,1-19,3,7,1,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2007.0,33.0,NaN,593,1-593
77,75,annurevisoci,32,1,2006,6,1-23,0,0,1,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2006.0,32.0,NaN,500,1-500
97,94,annurevisoci,31,1,2005,6,1-24,3,10,1,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2005.0,31.0,NaN,438,1-438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,3590,socsciresearch,31,3,2002,9,334-363,4,20,2,...,NaN,NaN,"Robison, KK; Crenshaw, EM",Post-industrial transformations and cyber-spac...,SOCIAL SCIENCE RESEARCH,2002.0,31.0,3.0,363,334-363
585,3688,socialforces,79,1,2000,9,1-28,2,8,2,...,NaN,NaN,"Tittle, CR; Rotolo, T",IQ and stratification: An empirical evaluation...,SOCIAL FORCES,2000.0,79.0,1.0,28,1-28
586,3697,socialforces,78,4,2000,6,1227-1276,2,16,1,...,NaN,NaN,"Rytina, S",Is occupational mobility declining in the US?,SOCIAL FORCES,2000.0,78.0,4.0,1276,1227-1276
591,3706,socimeth,35,0,2005,8,1-97,17,59,1,...,NaN,NaN,NaN,"Sociological Methodology 2005, Vol 35","SOCIOLOGICAL METHODOLOGY 2005, VOL 35",2005.0,35.0,NaN,348,1-348


In [49]:
#na_new["DI_y"]
na_new[na_new["DI_x"].isna()]
na_new.columns

Index(['artid', 'journal', 'volume', 'issue', 'year', 'month', 'pages_x',
       'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num', 'aut_cluster',
       'ack_cluster', 'com_cluster', 'bip_cluster', 'aut_jaccard',
       'ack_jaccard', 'com_jaccard', 'aut_density', 'ack_density',
       'com_density', 'bip_density', 'aut_avgdegree', 'ack_avgdegree',
       'com_avgdegree', 'bip_avgdegree', 'aut_flowhier', 'ack_flowhier',
       'com_flowhier', 'aut_numcomp', 'ack_numcomp', 'com_numcomp',
       'bip_numcomp', 'BP', 'EP_x', 'DI_x', 'D2_x', 'AU_x', 'TI_x', 'SO_x',
       'PY_x', 'VL_x', 'IS_x', 'EP_y', 'pages_y', 'DI_y', 'D2_y', 'AU_y',
       'TI_y', 'SO_y', 'PY_y', 'VL_y', 'IS_y', 'EP', 'pages'],
      dtype='object')

In [50]:
df_join["pages_diff"] = df_join['pages_y'] != df_join['pages_x']


df_join.loc[(df_join['pages_diff']) & (df_join['pages_y'].isna()), 'pages_y'] = df_join['pages_x']

df_join['pages_diff'] = df_join['pages_y'] != df_join['pages_x']

df_join



,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87-105,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127,False
3604,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167,False
3605,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137,False
3606,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False


In [51]:
df_join[df_join["DI"].isna()]


,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87-105,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578,3721,socimeth,30,0,2000,6,1-28,2,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-28,False
3579,3722,socimeth,30,0,2000,6,237-260,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237-260,False
3580,3723,socimeth,30,0,2000,6,261-315,0,12,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,261-315,False
3581,3724,socimeth,30,0,2000,6,165-202,2,73,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165-202,False


In [52]:
df_join_updated = df_join.merge(
    na_new[['year', 'volume', 'BP', 'DI_y']],  
    on=['year', 'volume', 'BP'],            
    how='left',                              
    suffixes=('', '_new')                   
)

df_join_updated['DI'] = df_join_updated['DI'].fillna(df_join_updated['DI_y'])

df_join_updated = df_join_updated.drop(columns=['DI_y'])

num_missing_doi_after_update = df_join_updated['DI'].isna().sum()
print(f"Total articles still missing DOI after update: {num_missing_doi_after_update}")

df_join_updated

Total articles still missing DOI after update: 311


,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
2,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
3,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
4,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3663,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127,False
3664,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167,False
3665,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137,False
3666,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False


In [53]:
df_join_updated = df_join_updated.drop_duplicates(subset=["year", "volume", "journal", "pages_x", "DI"], keep="first")

df_join_updated

,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
2,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
3,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
4,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3663,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127,False
3664,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167,False
3665,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137,False
3666,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False


In [54]:
df_join_updated[df_join_updated["DI"].isna()]

,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
28,28,annurevisoci,34,1,2008,6,1-13,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-13,False
51,50,annurevisoci,33,1,2007,6,1-19,3,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
77,75,annurevisoci,32,1,2006,6,1-23,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-23,False
97,94,annurevisoci,31,1,2005,6,1-24,3,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-24,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,3590,socsciresearch,31,3,2002,9,334-363,4,20,2,...,NaN,"Robison, KK; Crenshaw, EM",Post-industrial transformations and cyber-spac...,SOCIAL SCIENCE RESEARCH,2002.0,31.0,3.0,363,334-363,False
3604,3688,socialforces,79,1,2000,9,1-28,2,8,2,...,NaN,"Tittle, CR; Rotolo, T",IQ and stratification: An empirical evaluation...,SOCIAL FORCES,2000.0,79.0,1.0,28,1-28,False
3613,3697,socialforces,78,4,2000,6,1227-1276,2,16,1,...,NaN,"Rytina, S",Is occupational mobility declining in the US?,SOCIAL FORCES,2000.0,78.0,4.0,1276,1227-1276,False
3623,3706,socimeth,35,0,2005,8,1-97,17,59,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-97,False


In [55]:

# Manually mapped DOIs for missing articles
manual_doi_mapping = {
    ("annurevisoci", 34, 2008, '1'): "10.1146/annurev.soc.34.040507.134654",
    ("annurevisoci", 33, 2007, '1'): "10.1146/annurev.soc.33.040406.131720",
    ('annurevisoci', 32, 2006, '1'): "10.1146/annurev.soc.32.061604.123051",
    ('annurevisoci', 31, 2005, '1'): "10.1146/annurev.soc.31.041304.122209",
    ('amersocirevi', 67, 2002, '389'): "10.2307/3088963",
    ('amersocirevi', 69, 2004, '1'): "10.1177/000312240406900102",
    ('amersocirevi', 70, 2005, '136'): "10.2307/4145353",
    ('socnetworks', 28, 2006, '97'): "10.1016/j.socnet.2005.04.005",
    ('socialforces', 86, 2008, '1753'): "10.1353/sof.0.0038",
    ('socialforces', 86, 2008, '1781'): "10.1353/sof.0.0043",
    ('socialforces', 86, 2008, '1217'): "10.1353/sof.0.0008",
    ('socialforces', 86, 2008, '1189'): "10.1353/sof.0.0004",
    ('socialforces', 86, 2007, '417'): "10.1353/sof.2008.0022",
    ('socialforces', 86, 2008, '1481'): "10.1353/sof.0.0057",
    ('socialforces', 86, 2008, '1561'): "10.1353/sof.0.0040",
    ('socialforces', 86, 2008, '1529'): "10.1353/sof.0.0035",
    ('socialforces', 86, 2008, '1455'): "10.1353/sof.0.0052",
    ('socialforces', 86, 2008, '1367'): "10.1353/sof.0.0037",
    ('socialforces', 86, 2008, '1675'): "10.1353/sof.0.0059",
    ('socialforces', 86, 2008, '1805'): "10.1353/sof.0.0048",
    ('socialforces', 86, 2008, '1848'): "10.1353/sof.0.0056",
    ('socialforces', 86, 2008, '1273'): "10.1353/sof.0.0018",
    ('socialforces', 86, 2008, '995'): "10.1353/sof.0.0002",
    ('socialforces', 86, 2008, '911'): "10.1353/sof.0.0015",
    ('socialforces', 86, 2008, '1027'): "10.1353/sof.0.0027",
    ('socialforces', 86, 2008, '1344'): "10.1353/sof.0.0011",

    ('socialforces', 86, 2008, '937'): "10.1353/sof.0.0010",
    ('socialforces', 86, 2008, '1079'): "10.1353/sof.0.0017",
    ('socialforces', 84, 2005, '595'): "10.1353/sof.2005.0095",
    ('socialforces', 84, 2006, '1455'): "10.1353/sof.2006.0072",
    ('socialforces', 85, 2006, '145'): "10.1353/sof.2006.0126",
    ('socialforces', 83, 2005, '1065'): "10.1353/sof.2005.0040",
    ('social forces', 83, 2005, 1443): "",
    ('socialforces', 83, 2005, '1697'): "10.1353/sof.2005.0060",
    ('jmarriagefamily', 70, 2008, '466'): "10.1111/j.1741-3737.2008.00494.x",
    ('jmarriagefamily', 69, 2007, '1107'): "10.1111/j.1741-3737.2007.00435.x",
    ('jmarriagefamily', 68, 2006, '56'): "10.1111/j.1741-3737.2006.00233.x",
    ('jmarriagefamily', 67, 2005, '352'): "10.1111/j.0022-2445.2005.00121.x",
    ('jmarriagefamily', 67, 2005, '1'): "10.1111/j.0022-2445.2005.00001.x",
    ('jmarriagefamily', 67, 2005, '627'): "10.1111/j.1741-3737.2005.00158.x",
    ('jmarriagefamily', 67, 2005, '85'): "10.1111/j.0022-2445.2005.00007.x",
    ('jmarriagefamily', 66, 2004, '1042'): "10.1111/j.0022-2445.2004.00077.x",
    ('jmarriagefamily', 66, 2004, '821'): "10.1111/j.0022-2445.2004.00055.x",
    ('jmarriagefamily', 66, 2004, '1332'): "10.1111/j.0022-2445.2004.00096.x",
    ('jmarriagefamily', 66, 2004, '1300'): "10.1111/j.0022-2445.2004.00094.x",
    ('jmarriagefamily', 66, 2004, '1231'): "10.1111/j.0022-2445.2004.00089.x",
    ('jmarriagefamily', 66, 2004, '210'): "10.1111/j.0022-2445.2004.00014.x-i1",
    ('jmarriagefamily', 66, 2004, '318'): "10.1111/j.1741-3737.2004.00023.x",
    ('jmarriagefamily', 66, 2004, '515'): "10.1111/j.1741-3737.2004.00035.x",
    ('jmarriagefamily', 66, 2004, '163'): "10.1111/j.0022-2445.2004.00011.x-i1",
    ('jmarriagefamily', 66, 2004, '101'): "10.1111/1467-6478.00050-i1",
    ('jmarriagefamily', 66, 2004, '236'): "10.1111/j.0022-2445.2004.00015.x",
    ('jmarriagefamily', 65, 2003, '1088'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1082'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1081'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1085'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1087'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1084'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1083'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '767'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '764'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '766'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '763'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '765'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '768'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '762'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '501'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '500'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '502'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '504'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '1089'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '769'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '496'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '499'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '503'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '498'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 64, 2002, '270'): "10.1111/j.1741-3737.2002.00269.x",
    ('jmarriagefamily', 63, 2001, '804'): "10.1111/j.1741-3737.2001.00804.x",
    ('jmarriagefamily', 63, 2001, '582'): "10.1111/j.1741-3737.2001.00580.x",
    ('jmarriagefamily', 63, 2001, '589'): "10.1111/j.1741-3737.2001.00580.x",
    ('jmarriagefamily', 63, 2001, '592'): "10.1111/j.1741-3737.2001.00580.x",
    ('jmarriagefamily', 63, 2001, '585'): "10.1111/j.1741-3737.2001.00580.x",
    ('jmarriagefamily', 63, 2001, '279'): "10.1111/j.1741-3737.2001.00276.x",
    ('jmarriagefamily', 63, 2001, '1206'): "10.1111/j.1741-3737.2001.01199.x",
    ('jmarriagefamily', 63, 2001, '903'): "10.1111/j.1741-3737.2001.00902.x",
    ('socitheo', 22, 2004, '1'): "10.1111/j.1467-9558.2004.00199.x",
    ('socieduc', 80, 2007, '43'): "10.1177/0038040707080001",
    ('socieduc', 79, 2006, '1'): "10.1177/003804070607900101",
    ('socieduc', 78, 2005, '122'): "10.1177/003804070507800202",
    ('socieduc', 78, 2005, '1'): "10.1177/00380407050780010",
    
    ('socimeth', 36, 2006, '1'): "10.1111/j.1467-9531.2006.00172.x",
    ('socimeth', 37, 2007, '1'): "10.1111/j.1467-9531.2007.00184.x",
    ('socsciresearch', 32, 2003, '61'): "10.1016/S0049-089X(02)00013-3",
    ('socsciresearch', 32, 2003, '98'): "10.1016/S0049-089X(02)00021-2",
    ('socsciresearch', 31, 2002, '334'): "10.1016/S0049-089X(02)00004-2",

    ('socimeth', 35, 2005, '1'): "10.1111/j.0081-1750.2006.00164.x",
    ('socimeth', 30, 2000, '1'): "10.1111/0081-1750.00074",
    
    ('annurevisoci', 35, 2009, '1'): "10.1146/annurev-soc-070308-115936", 

    ('socialforces', 88, 2009, '945'): "10.1353/sof.0.0279", 
    ('socialforces', 88, 2009, '643'): "10.1353/sof.0.0262", 
    ('socialforces', 88, 2009, '785'): "10.1353/sof.0.0255", 
    ('socialforces', 88, 2009, '543'): "10.1353/sof.0.0278", 
    ('socialforces', 88, 2009, '723'): "10.1353/sof.0.0276", 
    ('socialforces', 88, 2009, '175'): "10.1353/sof.0.0231", 
    ('socialforces', 88, 2009, '147'): "10.1353/sof.0.0227", 
    ('socialforces', 88, 2009, '373'): "10.1353/sof.0.0234", 
    ('socialforces', 88, 2009, '425'): "10.1353/sof.0.0244", 
    ('socialforces', 88, 2009, '227'): "10.1353/sof.0.0241", 
    ('socialforces', 88, 2009, '519'): "10.1353/sof.0.0273", 
    ('socialforces', 88, 2009, '573'): "10.1353/sof.0.0252",
    ('socialforces', 88, 2009, '607'): "10.1353/sof.0.0257",
    ('socialforces', 88, 2009, '693'): "10.1353/sof.0.0271",
    ('socialforces', 88, 2009, '819'): "10.1353/sof.0.0260",
    ('socialforces', 88, 2009, '893'): "10.1353/sof.0.0269",
    ('socialforces', 88, 2009, '971'): "10.1353/sof.0.0253",
    ('socialforces', 88, 2009, '337'): "10.1353/sof.0.0229",
    ('socialforces', 88, 2009, '1'): "10.1353/sof.0.0233",
    ('socialforces', 88, 2009, '61'): "10.1353/sof.0.0243",
    ('socialforces', 88, 2009, '271'): "10.1353/sof.0.0221",
    ('socialforces', 88, 2009, '257'): "10.1353/sof.0.0246",
    ('socialforces', 88, 2009, '121'): "10.1353/sof.0.0223",

    ('socialforces', 87, 2009, '1943'): "10.1353/sof.0.0192", 
    ('socialforces', 87, 2009, '1499'): "10.1353/sof.0.0170", 
    ('socialforces', 87, 2009, '1481'): "10.1353/sof.0.0161", 
    ('socialforces', 87, 2009, '1355'): "10.1353/sof.0.0171", 
    ('socialforces', 87, 2009, '1649'): "10.1353/sof.0.0164", 
    ('socialforces', 87, 2009, '1873'): "10.1353/sof.0.0187",
    ('socialforces', 87, 2009, '2155'): "10.1353/sof.0.0218",
    ('socialforces', 87, 2009, '1743'): "10.1353/sof.0.0190",
    ('socialforces', 87, 2009, '1973'): "10.1353/sof.0.0191",
    ('socialforces', 87, 2009, '1913'): "10.1353/sof.0.0193",
    ('socialforces', 87, 2009, '2125'): "10.1353/sof.0.0219",
    ('socialforces', 87, 2009, '1997'): "10.1353/sof.0.0220",
    ('socialforces', 87, 2009, '2031'): "10.1353/sof.0.0189",
    ('socialforces', 87, 2009, '1321'): "10.1353/sof.0.0167",
    ('socialforces', 87, 2009, '1703'): "10.1353/sof.0.0162",
    ('socialforces', 87, 2009, '1529'): "10.1353/sof.0.0174",
    ('socialforces', 87, 2009, '1449'): "10.1353/sof.0.0157",
    ('socialforces', 87, 2009, '1489'): "10.1353/sof.0.0165",
    ('socialforces', 87, 2009, '1223'): "10.1353/sof.0.0177",
    ('socialforces', 87, 2009, '1389'): "10.1353/sof.0.0175",
    ('socialforces', 87, 2009, '1167'): "10.1353/sof.0.0169",
    ('socialforces', 87, 2008, '937'): "10.1353/sof.0.0138", 
    ('socialforces', 87, 2008, '1063'): "10.1353/sof.0.0136", 
    ('socialforces', 87, 2008, '741'): "10.1353/sof.0.0135", 
    ('socialforces', 87, 2008, '1158'): "10.1353/sof.0.0137", 
    ('socialforces', 87, 2008, '65'): "10.1353/sof.0.0095", 
    ('socialforces', 87, 2008, '299'): "10.1353/sof.0.0097", 
    ('socialforces', 87, 2008, '983'): "10.1353/sof.0.0149",
    ('socialforces', 87, 2008, '993'): "10.1353/sof.0.0155",
    ('socialforces', 87, 2008, '911'): "10.1353/sof.0.0133",
    ('socialforces', 87, 2008, '767'): "10.1353/sof.0.0140",
    ('socialforces', 87, 2008, '1039'): "10.1353/sof.0.0131",
    ('socialforces', 87, 2008, '887'): "10.1353/sof.0.0128",
    ('socialforces', 87, 2008, '713'): "10.1353/sof.0.0130",
    ('socialforces', 87, 2008, '795'): "10.1353/sof.0.0145",
    ('socialforces', 87, 2008, '1015'): "10.1353/sof.0.0126",
    ('socialforces', 87, 2008, '1089'): "10.1353/sof.0.0141",
    ('socialforces', 87, 2008, '1115'): "10.1353/sof.0.0146",
    ('socialforces', 87, 2008, '857'): "10.1353/sof.0.0123",
    ('socialforces', 87, 2008, '961'): "10.1353/sof.0.0143",
    ('socialforces', 87, 2008, '623'): "10.1353/sof.0.0147",
    ('socialforces', 87, 2008, '679'): "10.1353/sof.0.0125",
    ('socialforces', 87, 2008, '651'): "10.1353/sof.0.0153",
    ('socialforces', 87, 2008, '153'): "10.1353/sof.0.0065",
    ('socialforces', 87, 2008, '185'): "10.1353/sof.0.0074",
    ('socialforces', 87, 2008, '33'): "10.1353/sof.0.0086",
    ('socialforces', 87, 2008, '473'): "10.1353/sof.0.0076",
    ('socialforces', 87, 2008, '121'): "10.1353/sof.0.0104",
    ('socialforces', 87, 2008, '561'): "10.1353/sof.0.0088",
    ('socialforces', 87, 2008, '445'): "10.1353/sof.0.0068",
    ('socialforces', 87, 2008, '501'): "10.1353/sof.0.0122",
    ('socialforces', 87, 2008, '387'): "10.1353/sof.0.0079",
    
    ('socialforces', 79, 2000, '1'): "10.1093/sf/79.1.1",
    ('socialforces', 78, 2000, '1227'): "10.1093/sf/78.4.1195",
    ('socialforces', 78, 2000, '1133'): "10.1093/sf/78.3.1133",
    ('socialforces', 83, 2005, '1443'): "10.1353/sof.2005.0053", 
    ('socialforces', 86, 2007, '553'): "10.1093/sf/86.2.553", 

    ('jmarriagefamily', 62, 2000, '1018'): "10.1111/j.1741-3737.2000.01018.x",
    ('jmarriagefamily', 62, 2000, '1192'): "10.1111/j.1741-3737.2000.01192.x",
    ('jmarriagefamily', 62, 2000, '1247'): "10.1111/j.1741-3737.2000.01247.x",
    ('jmarriagefamily', 62, 2000, '911'): "10.1111/j.1741-3737.2000.00911.x",
    ('jmarriagefamily', 62, 2000, '1094'): "10.1111/j.1741-3737.2000.01094.x",
    ('jmarriagefamily', 62, 2000, '864'): "10.1111/j.1741-3737.2000.00847.x",
    ('jmarriagefamily', 62, 2000, '363'): "10.1111/j.1741-3737.2000.00363.x",
    ('jmarriagefamily', 62, 2000, '416'): "10.1111/j.1741-3737.2000.00416.x",
    ('jmarriagefamily', 62, 2000, '477'): "10.1111/j.1741-3737.2000.00477.x",
    ('jmarriagefamily', 62, 2000, '111'): "10.1111/j.1741-3737.2000.00111.x",
    ('jmarriagefamily', 62, 2000, '78'): "10.1111/j.1741-3737.2000.00078.x",
    ('jmarriagefamily', 62, 2000, '228'): "10.1111/j.1741-3737.2000.00228.x",
    ('jmarriagefamily', 70, 2008, '640'): "10.1111/j.1741-3737.2000.00640.x",
    ('jmarriagefamily', 62, 2000, '1136'): "10.1111/j.1741-3737.2000.01136.x",
    ('jmarriagefamily', 62, 2000, '1288'): "10.1111/j.1741-3737.2000.01288.x",
    ('jmarriagefamily', 62, 2000, '1208'): "10.1111/j.1741-3737.2000.01208.x",
    ('jmarriagefamily', 62, 2000, '1234'): "10.1111/j.1741-3737.2000.01234.x",
    ('jmarriagefamily', 62, 2000, '896'): "10.1111/j.1741-3737.2000.00896.x",
    ('jmarriagefamily', 62, 2000, '849'): "10.1111/j.1741-3737.2000.00847.x",
    ('jmarriagefamily', 62, 2000, '577'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '322'): "10.1111/j.1741-3737.2000.00322.x",
    ('jmarriagefamily', 62, 2000, '399'): "10.1111/j.1741-3737.2000.00399.x",
    ('jmarriagefamily', 62, 2000, '520'): "10.1111/j.1741-3737.2000.00520.x",
    ('jmarriagefamily', 62, 2000, '349'): "10.1111/j.1741-3737.2000.00349.x",
    ('jmarriagefamily', 62, 2000, '508'): "10.1111/j.1741-3737.2000.00508.x",
    ('jmarriagefamily', 62, 2000, '283'): "10.1111/j.1741-3737.2000.00283.x",
    ('jmarriagefamily', 62, 2000, '489'): "10.1111/j.1741-3737.2000.00489.x",
    ('jmarriagefamily', 62, 2000, '465'): "10.1111/j.1741-3737.2000.00465.x",
    ('jmarriagefamily', 62, 2000, '388'): "10.1111/j.1741-3737.2000.00388.x",
    ('jmarriagefamily', 62, 2000, '449'): "10.1111/j.1741-3737.2000.00449.x",
    ('jmarriagefamily', 62, 2000, '430'): "10.1111/j.1741-3737.2000.00430.x",
    ('jmarriagefamily', 62, 2000, '582'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '375'): "10.1111/j.1741-3737.2000.00375.x",
    ('jmarriagefamily', 62, 2000, '585'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '588'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '592'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '570'): "10.1111/j.1741-3737.2000.00570.x",
    ('jmarriagefamily', 62, 2000, '563'): "10.1111/j.1741-3737.2000.00563.x",
    ('jmarriagefamily', 62, 2000, '93'): "10.1111/j.1741-3737.2000.00093.x",
    ('jmarriagefamily', 62, 2000, '123'): "10.1111/j.1741-3737.2000.00123.x",
    ('jmarriagefamily', 62, 2000, '61'): "10.1111/j.1741-3737.2000.00061.x",
    ('jmarriagefamily', 62, 2000, '48'): "10.1111/j.1741-3737.2000.00048.x",
    ('jmarriagefamily', 62, 2000, '256'): "10.1111/j.1741-3737.2000.00256.x",
    ('jmarriagefamily', 62, 2000, '147'): "10.1111/j.1741-3737.2000.00147.x",
    ('jmarriagefamily', 62, 2000, '175'): "10.1111/j.1741-3737.2000.00175.x",
    ('jmarriagefamily', 62, 2000, '162'): "10.1111/j.1741-3737.2000.00162.x",

    ('socieduc', 76, 2003, '368'): "", # not an article, didn't know how to proceed
    ('socieduc', 76, 2003, '189'): "", # not an article, didn't know how to proceed
    ('socieduc', 75, 2002, '97'): "", # not an article (in memoriam), didn't know how to proceed
    ('socieduc', 73, 2000, '92'): "10.2307/2673239",
    ('socieduc', 74, 2001, '135'): "10.2307/2673258",
    ('socieduc', 74, 2001, '100'): "10.2307/2673256",
    ('socieduc', 74, 2001, '115'): "10.2307/2673257",
    ('socieduc', 74, 2001, '3'): "10.2307/2673250",
    ('socieduc', 74, 2001, '35'): "10.2307/2673252",
    ('socieduc', 74, 2001, '71'): "10.2307/2673254",
    ('socieduc', 74, 2001, '159'): "10.2307/2673260",
    ('socieduc', 74, 2001, '154'): "10.2307/2673259",
    ('socieduc', 74, 2001, '178'): "10.2307/2673262",
    ('socieduc', 74, 2001, '50'): "10.2307/2673253",
    ('socieduc', 74, 2001, '169'): "10.2307/2673261",
    ('socieduc', 74, 2001, '19'): "10.2307/2673251",
    ('socieduc', 74, 2001, '88'): "10.2307/2673255",

    ('socimeth', 39, 2009, '1'): "10.1111/j.1467-9531.2009.01214.x",
    ('socimeth', 38, 2008, '1'): "10.1111/j.1467-9531.2008.00199.x",

    ('socsciresearch', 38, 2009, '1'): "10.1016/j.ssresearch.2008.03.007",
    ('socsciresearch', 38, 2009, '19'): "10.1016/j.ssresearch.2008.05.003",
    ('socsciresearch', 38, 2009, '29'): "10.1016/j.ssresearch.2008.07.004",
    ('socsciresearch', 38, 2009, '39'): "10.1016/j.ssresearch.2008.09.004",
    ('socsciresearch', 38, 2009, '55'): "10.1016/j.ssresearch.2008.10.002",
    ('socsciresearch', 38, 2009, '71'): "10.1016/j.ssresearch.2008.10.001",
    ('socsciresearch', 38, 2009, '86'): "10.1016/j.ssresearch.2008.08.003",
    ('socsciresearch', 38, 2009, '103'): "10.1016/j.ssresearch.2008.08.005",
    ('socsciresearch', 38, 2009, '118'): "10.1016/j.ssresearch.2008.07.002",
    ('socsciresearch', 38, 2009, '134'): "10.1016/j.ssresearch.2008.09.001",
    ('socsciresearch', 38, 2009, '146'): "10.1016/j.ssresearch.2008.08.001"
}


# Function to manually add DOIs
def add_manual_dois(row):
    key = (row["journal"], row["volume"], row["year"], row["BP"])
    return manual_doi_mapping.get(key, row["DI"])  # Keep original DOI if already present

# Apply manual mapping to the missing DOIs
df_join_updated["DI"] = df_join_updated.apply(add_manual_dois, axis=1)

num_missing_doi_after_fix = df_join_updated["DI"].isna().sum() + len(df_join_updated[df_join_updated["DI"] == ""])
print(f"Total articles still missing DOI after manual fix: {num_missing_doi_after_fix}")

missing_doi_tuples = df_join_updated[df_join_updated["DI"] == ""][["journal", "volume", "year", "BP"]]

for row in missing_doi_tuples.itertuples(index=False, name=None):
    print(row)  # Prints as tuple (journal, volume, year, beginning_page)



Total articles still missing DOI after manual fix: 3
('socieduc', 76, 2003, '368')
('socieduc', 76, 2003, '189')
('socieduc', 75, 2002, '97')


<ipython-input-55-d4c01ad7592e>:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_join_updated["DI"] = df_join_updated.apply(add_manual_dois, axis=1)


In [56]:
disruption_file = "dataverse_files/Aggregated_20210521.txt"

disruption_df = pd.read_csv(disruption_file, 
                           sep='\t',  # adjust separator if needed
                           names=['MAGpaperID', 'year', 'field', 'teamsize', 'citation', 'disruption'])

# Display the first few rows to verify the data
print("First few rows of the disruption dataset:")
display(disruption_df.head())

print("\nDataset info:")
display(disruption_df.info())


First few rows of the disruption dataset:


,MAGpaperID,year,field,teamsize,citation,disruption
0,1970392578,1830,10,1,3,0.750000
1,2108276706,1842,5,1,1,0.333333
2,2022566795,1846,7,1,12,0.200000
3,2065789632,1850,9,1,3,0.150000
4,219463075,1851,5,1,3,0.214286



Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19495814 entries, 0 to 19495813
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   MAGpaperID  int64  
 1   year        int64  
 2   field       int64  
 3   teamsize    int64  
 4   citation    int64  
 5   disruption  float64
dtypes: float64(1), int64(5)
memory usage: 892.4 MB


None